In [2]:
import pandas as pd

In [2]:
test_df = pd.read_csv("test.csv")

In [3]:
test_df

,label,path,split,is_query,is_gallery
0,0,data/test_public/images/00000000/0.jpg,test,True,False
1,0,data/test_public/images/00000000/1.jpg,test,False,True
2,1,data/test_public/images/00000001/0.jpg,test,True,False
3,1,data/test_public/images/00000001/1.jpg,test,False,True
4,2,data/test_public/images/00000002/0.jpg,test,True,False
...,...,...,...,...,...
322377,161188,data/test_public/images/00161188/1.jpg,test,False,True
322378,161189,data/test_public/images/00161189/0.jpg,test,True,False
322379,161189,data/test_public/images/00161189/1.jpg,test,False,True
322380,161190,data/test_public/images/00161190/0.jpg,test,True,False


In [3]:
val_df = pd.read_csv("val.csv")
val_df.head()

,label,path,split,is_query,is_gallery
0,0,data/train/images/000000/6.jpg,validation,True,False
1,0,data/train/images/000000/7.jpg,validation,False,True
2,0,data/train/images/000000/8.jpg,validation,False,True
3,1,data/train/images/000001/7.jpg,validation,True,False
4,1,data/train/images/000001/8.jpg,validation,False,True


In [5]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       30000 non-null  int64 
 1   path        30000 non-null  object
 2   split       30000 non-null  object
 3   is_query    30000 non-null  bool  
 4   is_gallery  30000 non-null  bool  
dtypes: bool(2), int64(1), object(2)
memory usage: 761.8+ KB


In [6]:
from predict import get_query_gallery_pairs

/kaggle/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
val_df = get_query_gallery_pairs(val_df)
val_df.head(20)

,label,path,is_query,is_gallery
0,0,data/train/images/000000/6.jpg,True,False
1,0,data/train/images/000000/7.jpg,False,True
2,0,data/train/images/000000/6.jpg,True,False
3,0,data/train/images/000000/8.jpg,False,True
4,1,data/train/images/000001/7.jpg,True,False
5,1,data/train/images/000001/8.jpg,False,True
6,1,data/train/images/000001/7.jpg,True,False
7,1,data/train/images/000001/9.jpg,False,True
8,2,data/train/images/000002/7.jpg,True,False
9,2,data/train/images/000002/8.jpg,False,True


In [8]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       40000 non-null  int64 
 1   path        40000 non-null  object
 2   is_query    40000 non-null  bool  
 3   is_gallery  40000 non-null  bool  
dtypes: bool(2), int64(1), object(1)
memory usage: 703.2+ KB


In [10]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

train_df

,label,path,split,is_query,is_gallery
0,0,data/train/images/000000/0.jpg,train,NaN,NaN
1,0,data/train/images/000000/1.jpg,train,NaN,NaN
2,0,data/train/images/000000/2.jpg,train,NaN,NaN
3,0,data/train/images/000000/3.jpg,train,NaN,NaN
4,0,data/train/images/000000/4.jpg,train,NaN,NaN
...,...,...,...,...,...
67728,9999,data/train/images/009999/1.jpg,train,NaN,NaN
67729,9999,data/train/images/009999/2.jpg,train,NaN,NaN
67730,9999,data/train/images/009999/3.jpg,train,NaN,NaN
67731,9999,data/train/images/009999/4.jpg,train,NaN,NaN


In [15]:
pd.concat([train_df, val_df]).to_csv('full_data.csv', index=False)